In [1]:
import pyaudio
import numpy as np
from queue import Queue
import threading
import atexit
import sounddevice as sd

In [2]:
# Constants for audio stream configuration
FORMAT = pyaudio.paInt16  # Sample format (16-bit)
CHANNELS = 1             # Number of audio channels (1 for mono, 2 for stereo)
RATE = 44100             # Sample rate (samples per second)
CHUNK_SIZE = 44100        # Number of audio frames per buffer

DETECT_THRESHOLD = 800   # Threshold to detect a clap
DETECT_DURATION = 0.5    # Time in seconds to detect a clap
RECORD_DURATION = 1      # Time in seconds to record audio after clap is detected

In [3]:
audio_input_queue = Queue(maxsize=2)
stop_record_event = threading.Event()


In [4]:
stop_record_event.clear()

# Function to capture audio data
def capture_audio_data():
    global stop_record_event
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK_SIZE)

    try:
        while not stop_record_event.is_set():
            # Read audio data from the stream
            data = stream.read(int(DETECT_DURATION * RATE))
            # Process the audio data (you can replace this with your custom processing)
            audio_data = np.frombuffer(data, dtype=np.int16)

            # audio_data = sd.rec(int(DETECT_DURATION * RATE), samplerate=RATE, channels=1, dtype=np.int16)
            # sd.wait()
            # audio_data = np.concatenate(audio_data)

            print(audio_data[:20])
            print("detecting:", np.max(audio_data))
            
            if np.max(audio_data) < DETECT_THRESHOLD:
                continue
            
            flag_first = True
            while True:
                data = stream.read(int(RECORD_DURATION * RATE))
                # print(data[:20])
                audio_data = np.frombuffer(data, dtype=np.int16)

                # audio_data = sd.rec(int(RECORD_DURATION * RATE), samplerate=RATE, channels=1, dtype=np.int16)
                # sd.wait()
                # audio_data = np.concatenate(audio_data)

                if not flag_first and np.max(audio_data) < DETECT_THRESHOLD:
                    break
                flag_first = False

                # print(audio_data[:20])
                # # Put audio data into the queue
                audio_input_queue.put(audio_data)
                # print(np.max(audio_data))
                # sd.play(audio_data, samplerate=RATE)
                # sd.wait()


    except KeyboardInterrupt:
        print("Recording stopped.")
    finally:
        # Close the audio stream and terminate PyAudio
        stream.stop_stream()
        stream.close()
        audio.terminate()

# Create and start the audio capture thread
audio_thread = threading.Thread(target=capture_audio_data)
audio_thread.start()

# Register a function to stop the audio thread at program exit using atexit
def stop_audio_thread():
    global stop_record_event
    stop_record_event.set()
    audio_thread.join()

atexit.register(stop_audio_thread)



<function __main__.stop_audio_thread()>

In [5]:
# stop_audio_thread()

In [6]:


# # Retrieve items from the queue
# while not audio_input_queue.empty():
#     audio_input = audio_input_queue.get()
#     print(np.max(audio_input))  
#     sd.play(audio_input, samplerate=RATE)
#     # sd.wait()

# # sd.wait()
# print("last=",np.max(audio_input))  
# sd.play(audio_input, samplerate=RATE)
# sd.wait()


In [7]:
from pynput import keyboard

def on_press(key):
    if key == keyboard.Key.esc:
        # Stop listener
        return False

def on_release(key):
    pass

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()

In [8]:
# audio_input = audio_input_queue.get()

# sd.play(audio_input, samplerate=RATE)
# # sd.wait()

while True:
    if not audio_input_queue.empty():
        audio_input = audio_input_queue.get()
        sd.play(audio_input, samplerate=RATE)
        sd.wait()
    if not listener.running:
        break

[ 0  0 -1  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0]
detecting: 1
[0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
detecting: 1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
detecting: 1
[ 0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0 -1  0  0  0  1]
detecting: 1
[ 0  0  0  0  0  0 -1  0 -1  1  0  1  1  0  0  0  0  0  0  0]
detecting: 1
[ 0  0  0  0  0 -1  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
detecting: 1
[ 0  0  0  0  0 -1 -1 -1 -1  0  0  0  0  0 -1  0  0  0  0  0]
detecting: 1
[1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
detecting: 1
[ 0  0 -1  0  0  0 -1 -1  0  0  1  0 -1  0  0  0  0  0  0 -1]
detecting: 1
[ 0 -1  1 -1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
detecting: 1
[ 1  0  0 -1  0 -1  1  0  1  0  0  0  0  1 -1  0  0  0  0  0]
detecting: 1
[ 0  1  0  0  0  0  0  0  0  0  0 -1  0  0  1  0  0  0  0  0]
detecting: 1
[ 0  0  0 -1  0  0  0  1  0  1  0 -1  0  1  0  0 -1  0  0  0]
detecting: 1
[ 0  0  0  0  0  0 -1  1  0  1  0  0  1  0  0  0  0  0  1 -1]
detecting: 286
[ 0  0  

KeyboardInterrupt: 

[ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  1  0]
detecting: 1
[ 0  0  0  0  0  0  0  0 -1  0  0  0 -1  0 -1  0  0  0  0  0]
detecting: 1
[0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1]
detecting: 1
[ 0 -1  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  1  1 -1  0]
detecting: 1
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0 -1  0  0  0  0]
detecting: 1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
detecting: 1
[-1  0  0  0  0  0 -1  0 -1  0  1  0  0  1  0  0 -1  0 -1  0]
detecting: 1
[ 0  0  0  0  0  0  1  0  0 -1  0  0  0  0  0  0  0  0  0  0]
detecting: 1
[ 0  0  0  0  0 -1 -1  0  0 -1  1  0  1  0  0  0  0  0  0  1]
detecting: 1
[ 0  0  0  0  0  0  0  1  0  0  0  0 -1  1  0  0  0  0  0  0]
detecting: 1
[-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
detecting: 1
[ 1  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  1  0  0]
detecting: 35
[ 0 -1 -2 -1 -1 -1  0  1  1  2  3  4  4  5  7  7  9 10 11 12]
detecting: 29
[0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 1]
detecting: 2
[-1  0  